# Tutoriel DSPy et GEPA : Classification Automatique de Tickets IT

Ce tutoriel complet vous guide de manière progressive à travers **DSPy** (Declarative Self-improving Language Programs) et **GEPA** (Generalized Error-driven Prompt Augmentation).

## Objectifs d'apprentissage

1. Maîtriser les **Signatures** : définir les entrées/sorties de vos tâches
2. Comprendre les **Modules** : les différents types et comment les composer
3. Évaluer les performances avec des **métriques**
4. Optimiser automatiquement avec les **Optimiseurs**
5. Utiliser différents **modèles de langage** facilement
6. Maîtriser **GEPA** pour l'optimisation avancée

## Prérequis

- Python 3.9+
- Ollama installé et en cours d'exécution
- Un modèle téléchargé (ex: `ollama pull llama3.1:8b`)

## Structure du tutoriel

1. **Configuration et Données** - Préparer l'environnement
2. **Les Signatures DSPy** - Définir ce que vous voulez faire
3. **Les Modules DSPy** - Comment le faire
4. **L'Évaluation** - Mesurer les performances
5. **Les Optimiseurs** - Améliorer automatiquement
6. **Multi-Modèles** - Flexibilité entre providers
7. **GEPA en Pratique** - Optimisation avancée

## Configuration initiale

Commençons par installer les dépendances et configurer notre environnement.

In [ ]:
# Installation des dépendances (décommentez si nécessaire)
# !uv pip install dspy-ai

In [39]:
import dspy
import warnings
warnings.filterwarnings('ignore')

print("✅ Bibliothèques importées avec succès!")

✅ Bibliothèques importées avec succès!


### Configuration du modèle de langage

Nous utilisons Ollama avec Llama 3.1, un modèle open-source qui fonctionne localement.

In [40]:
print("🚀 Configuration de DSPy avec Ollama...")

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

print("✅ DSPy configuré avec Ollama Llama 3.1:8b")

🚀 Configuration de DSPy avec Ollama...
✅ DSPy configuré avec Ollama Llama 3.1:8b


### Préparation des données

Nous travaillons avec des tickets IT en français. Chaque ticket contient :
- Une **description** du problème
- Une **catégorie** (Hardware, Software, Network, etc.)
- Une **priorité** (Low, Medium, High, Urgent, Critical)

In [1]:
# Données d'entraînement
trainset = [
    {"ticket": "Mon ordinateur ne démarre plus depuis ce matin. J'ai une présentation importante dans 2 heures.", "category": "Hardware", "priority": "Urgent"},
    {"ticket": "Je n'arrive pas à me connecter à l'imprimante du 3e étage. Ça peut attendre.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le VPN ne fonctionne plus. Impossible d'accéder aux fichiers du serveur.", "category": "Network", "priority": "High"},
    {"ticket": "J'ai oublié mon mot de passe Outlook. Je peux utiliser le webmail.", "category": "Account", "priority": "Medium"},
    {"ticket": "Le site web affiche une erreur 500. Les clients ne peuvent plus commander!", "category": "Application", "priority": "Critical"},
    {"ticket": "Ma souris sans fil ne répond plus bien. Les piles sont faibles.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le système de paie ne calcule pas les heures supplémentaires. C'est la fin du mois.", "category": "Application", "priority": "Urgent"},
    {"ticket": "J'aimerais une mise à jour de mon logiciel Adobe quand vous aurez le temps.", "category": "Software", "priority": "Low"},
    {"ticket": "Le serveur de base de données est très lent. Toute la production est impactée.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "Je ne reçois plus les emails. J'attends des réponses de fournisseurs.", "category": "Email", "priority": "High"},
    {"ticket": "Mon écran externe ne s'affiche plus. Je peux travailler sur le laptop.", "category": "Hardware", "priority": "Medium"},
    {"ticket": "Le wifi de la salle A ne fonctionne pas. Réunion avec des externes dans 30 min.", "category": "Network", "priority": "Urgent"},
    {"ticket": "Je voudrais installer Slack pour mieux collaborer avec l'équipe.", "category": "Software", "priority": "Medium"},
    {"ticket": "Le système de sauvegarde a échoué cette nuit selon le rapport.", "category": "Infrastructure", "priority": "High"},
    {"ticket": "Mon clavier a une touche qui colle. C'est gérable mais ennuyeux.", "category": "Peripherals", "priority": "Low"}
]

# Données de validation
valset = [
    {"ticket": "Le serveur de fichiers est inaccessible. Personne ne peut travailler.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.", "category": "Account", "priority": "Urgent"},
    {"ticket": "L'écran de mon collègue en vacances clignote. On peut attendre.", "category": "Hardware", "priority": "Low"},
    {"ticket": "Le CRM plante quand j'essaie d'exporter les contacts.", "category": "Application", "priority": "High"},
    {"ticket": "Je voudrais changer ma photo de profil quand vous aurez un moment.", "category": "Account", "priority": "Low"},
    {"ticket": "La vidéoconférence ne fonctionne pas. Réunion avec New York dans 10 minutes!", "category": "Application", "priority": "Critical"},
    {"ticket": "Mon antivirus affiche un message d'expiration mais tout fonctionne.", "category": "Software", "priority": "Medium"}
]

# Catégories et priorités possibles
CATEGORIES = ["Hardware", "Software", "Network", "Application", "Infrastructure", "Account", "Email", "Peripherals"]
PRIORITIES = ["Low", "Medium", "High", "Urgent", "Critical"]

print(f"📊 Données chargées : {len(trainset)} entraînement, {len(valset)} validation")
print(f"📦 {len(CATEGORIES)} catégories, {len(PRIORITIES)} priorités")

📊 Données chargées : 15 entraînement, 7 validation
📦 8 catégories, 5 priorités
